# Desafio 6

Neste desafio, vamos praticar _feature engineering_, um dos processos mais importantes e trabalhosos de ML. Utilizaremos o _data set_ [Countries of the world](https://www.kaggle.com/fernandol/countries-of-the-world), que contém dados sobre os 227 países do mundo com informações sobre tamanho da população, área, imigração e setores de produção.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [17]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()

In [19]:
countries = pd.read_csv("countries.csv", decimal=',')

In [31]:
new_column_names = [
    "Country", "Region", "Population", "Area", "Pop_density", "Coastline_ratio",
    "Net_migration", "Infant_mortality", "GDP", "Literacy", "Phones_per_1000",
    "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", "Agriculture",
    "Industry", "Service"
]

countries.columns = new_column_names

countries.head(5)

## Observações

Esse _data set_ ainda precisa de alguns ajustes iniciais. Primeiro, note que as variáveis numéricas estão usando vírgula como separador decimal e estão codificadas como strings. Corrija isso antes de continuar: transforme essas variáveis em numéricas adequadamente.

Além disso, as variáveis `Country` e `Region` possuem espaços a mais no começo e no final da string. Você pode utilizar o método `str.strip()` para remover esses espaços.

## Inicia sua análise a partir daqui

In [21]:
# Sua análise começa aqui.
countries['Country'] = countries['Country'].str.strip(' ')
countries['Region'] = countries['Region'].str.strip(' ')

## Questão 1

Quais são as regiões (variável `Region`) presentes no _data set_? Retorne uma lista com as regiões únicas do _data set_ com os espaços à frente e atrás da string removidos (mas mantenha pontuação: ponto, hífen etc) e ordenadas em ordem alfabética.

In [22]:
def q1():
    # Retorne aqui o resultado da questão 1.
    return list(countries['Region'].str.strip('').value_counts().sort_index(ascending=True).index)

## Questão 2

Discretizando a variável `Pop_density` em 10 intervalos com `KBinsDiscretizer`, seguindo o encode `ordinal` e estratégia `quantile`, quantos países se encontram acima do 90º percentil? Responda como um único escalar inteiro.

In [23]:
def q2():
    # Retorne aqui o resultado da questão 2.
    est = KBinsDiscretizer(10, encode='ordinal', strategy='quantile')
    disc = est.fit_transform(countries.Pop_density.values.reshape(-1, 1))
    quant = np.quantile(disc, 0.9)
    return int(len(disc[disc > quant]))

# Questão 3

Se codificarmos as variáveis `Region` e `Climate` usando _one-hot encoding_, quantos novos atributos seriam criados? Responda como um único escalar.

In [24]:
def q3():
    # Retorne aqui o resultado da questão 3.
    countries.Climate.fillna(0, inplace=True)
    onehot = OneHotEncoder(sparse=False)
    encoded = onehot.fit_transform(countries[['Region', 'Climate']])
    return int(encoded.shape[1])

## Questão 4

Aplique o seguinte _pipeline_:

1. Preencha as variáveis do tipo `int64` e `float64` com suas respectivas medianas.
2. Padronize essas variáveis.

Após aplicado o _pipeline_ descrito acima aos dados (somente nas variáveis dos tipos especificados), aplique o mesmo _pipeline_ (ou `ColumnTransformer`) ao dado abaixo. Qual o valor da variável `Arable` após o _pipeline_? Responda como um único float arredondado para três casas decimais.

In [25]:
test_country = [
    'Test Country', 'NEAR EAST', -0.19032480757326514,
    -0.3232636124824411, -0.04421734470810142, -0.27528113360605316,
    0.13255850810281325, -0.8054845935643491, 1.0119784924248225,
    0.6189182532646624, 1.0074863283776458, 0.20239896852403538,
    -0.043678728558593366, -0.13929748680369286, 1.3163604645710438,
    -0.3699637766938669, -0.6149300604558857, -0.854369594993175,
    0.263445277972641, 0.5712416961268142
]

In [26]:
def q4():
    # Retorne aqui o resultado da questão 4.
    numeric = countries.select_dtypes(include=['float64','int64'])
    pipeline = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), 
                            ('scaler', StandardScaler())
                            ])
    numeric_data = pipeline.fit(numeric)
    test_array = np.array(test_country[2:]).reshape(1, -1)
    test_transformed = pipeline.transform(test_array)
    df = pd.DataFrame(columns=numeric.columns,
            data=test_transformed)
    return round(float(df.Arable.values), 3)

## Questão 5

Descubra o número de _outliers_ da variável `Net_migration` segundo o método do _boxplot_, ou seja, usando a lógica:

$$x \notin [Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}] \Rightarrow x \text{ é outlier}$$

que se encontram no grupo inferior e no grupo superior.

Você deveria remover da análise as observações consideradas _outliers_ segundo esse método? Responda como uma tupla de três elementos `(outliers_abaixo, outliers_acima, removeria?)` ((int, int, bool)).

In [27]:
def q5():
    # Retorne aqui o resultado da questão 4.
    q75 = countries.Net_migration.describe()[6]
    q25 = countries.Net_migration.describe()[4]
    iqr = q75 - q25
    upper_lim = q75 + iqr * 1.5
    down_lim = q25 - iqr * 1.5
    downs = len(countries[countries.Net_migration < down_lim]['Net_migration'])
    ups = len(countries[countries.Net_migration > upper_lim]['Net_migration'])
    return tuple((downs, ups, False))

## Questão 6
Para as questões 6 e 7 utilize a biblioteca `fetch_20newsgroups` de datasets de test do `sklearn`

Considere carregar as seguintes categorias e o dataset `newsgroups`:

```
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
```


Aplique `CountVectorizer` ao _data set_ `newsgroups` e descubra o número de vezes que a palavra _phone_ aparece no corpus. Responda como um único escalar.

In [28]:
from sklearn.datasets import fetch_20newsgroups
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)

In [29]:
def q6():
    # Retorne aqui o resultado da questão 4.
    from sklearn.feature_extraction.text import CountVectorizer
    cv = CountVectorizer()
    matrix = cv.fit_transform(newsgroup.data)
    get_phone = cv.vocabulary_.get('phone')
    return int(matrix[:, get_phone].sum())

## Questão 7

Aplique `TfidfVectorizer` ao _data set_ `newsgroups` e descubra o TF-IDF da palavra _phone_. Responda como um único escalar arredondado para três casas decimais.

In [30]:
def q7():
    # Retorne aqui o resultado da questão 4.
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer()
    tf = vectorizer.fit_transform(newsgroup.data)
    get_phone = vectorizer.get_feature_names().index('phone')
    return round(float(tf[:, get_phone].sum()), 3)